In [1]:
!ls data

chicago_taxi_trips_2016_01.csv	chicago_taxi_trips_2016_09.csv
chicago_taxi_trips_2016_02.csv	chicago_taxi_trips_2016_10.csv
chicago_taxi_trips_2016_03.csv	chicago_taxi_trips_2016_11.csv
chicago_taxi_trips_2016_04.csv	chicago_taxi_trips_2016_12.csv
chicago_taxi_trips_2016_05.csv	column_remapping.json
chicago_taxi_trips_2016_06.csv	company_dedupe_map.p
chicago_taxi_trips_2016_07.csv	data_dictionary.csv
chicago_taxi_trips_2016_08.csv


In [13]:
import pandas as pd
import numpy as np
import json
import pickle

In [3]:
trips = pd.read_csv('data/chicago_taxi_trips_2016_01.csv')

mapping = pd.read_json('data/column_remapping.json')

with open('data/company_dedupe_map.p', 'rb') as f:
    company_dedupe_map = pickle.load(f)

In [4]:
def prepare(trips):
    # drop zero fares
    n = trips.shape[0]
    trips = trips.dropna(subset=['fare'])
    print(f'{n - trips.shape[0]} rows with 0 fare were dropped')
    
    # drop zero trip_miles
    n = trips.shape[0]
    trips = trips[trips.trip_miles > 0]
    print(f'{n - trips.shape[0]} rows with 0 trip_miles were dropped')
    
    # drop na geo coordinates
    n = trips.shape[0]
    trips = trips.dropna(subset=['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'])
    print(f'{n - trips.shape[0]} rows with na geo coordinates were dropped')
    
    # restore real longitude/latitude
    trips['pickup_latitude'] = trips['pickup_latitude'].map(lambda x: mapping.pickup_latitude[x])
    trips['pickup_longitude'] = trips['pickup_longitude'].map(lambda x: mapping.pickup_longitude[x])
    trips['dropoff_latitude'] = trips['dropoff_latitude'].map(lambda x: mapping.dropoff_latitude[x])
    trips['dropoff_longitude'] = trips['dropoff_longitude'].map(lambda x: mapping.dropoff_longitude[x])
    
    # remove duplicates in company names (taken from https://www.kaggle.com/sohier/taxi-revenues-eda)
    trips.company = trips.company.map(company_dedupe_map, na_action='ignore')
    
    # restore company names
    trips.company = trips.company.map(lambda x: mapping['company'][x], na_action='ignore')

    return trips

In [5]:
all_trips = []

# let's keep only the first two months
for i in range(1,3):
    fname = f'data/chicago_taxi_trips_2016_{i:02d}.csv'
    print(f'Reading {fname}')
    df = pd.read_csv(fname)
    all_trips.append(prepare(df))

Reading data/chicago_taxi_trips_2016_01.csv
33 rows with 0 fare were dropped
450266 rows with 0 trip_miles were dropped
160271 rows with na geo coordinates were dropped
Reading data/chicago_taxi_trips_2016_02.csv
10 rows with 0 fare were dropped
458117 rows with 0 trip_miles were dropped
161625 rows with na geo coordinates were dropped


In [6]:
trips_2months = pd.concat(all_trips[:2], axis=0)

In [7]:
trips_2months.to_csv('taxi_trips_2months.csv.gz', compression='gzip')

In [8]:
trips_2months.head()

,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,85.0,2016-1-13 06:15:00,2016-1-13 06:15:00,180.0,0.4,NaN,NaN,24.0,24.0,4.50,0.0,0.0,0.0,4.50,Cash,Taxi Affiliation Services,41.901207,-87.676356,41.901207,-87.676356
3,4237.0,2016-1-23 17:30:00,2016-1-23 17:30:00,480.0,1.1,NaN,NaN,6.0,6.0,7.00,0.0,0.0,0.0,7.00,Cash,NaN,41.944227,-87.655998,41.944227,-87.655998
5,1987.0,2016-1-8 18:15:00,2016-1-8 18:45:00,1080.0,6.2,NaN,NaN,8.0,3.0,17.75,0.0,0.0,0.0,17.75,Cash,NaN,41.899602,-87.633308,41.965812,-87.655879
7,6400.0,2016-1-26 04:15:00,2016-1-26 04:15:00,60.0,0.2,NaN,NaN,16.0,16.0,3.75,0.0,0.0,0.0,3.75,Cash,Taxi Affiliation Services,41.953582,-87.723452,41.953582,-87.723452
10,1078.0,2016-1-25 09:00:00,2016-1-25 09:00:00,480.0,1.3,NaN,792.0,8.0,8.0,7.00,1.0,0.0,0.0,8.00,Credit Card,Choice Taxi Association,41.898332,-87.620763,41.893216,-87.637844


In [9]:
trips_2months.shape

(2226675, 20)

In [11]:
trips_2months.taxi_id.nunique()

3537

In [15]:
sampled_taxi_ids = np.random.choice(trips_2months.taxi_id.unique(), size=500, replace=False)

In [16]:
trips_2months_sample = trips_2months[trips_2months.taxi_id.isin(sampled_taxi_ids)]

In [19]:
trips_2months_sample.shape

(311151, 20)

In [20]:
trips_2months_sample.to_csv('taxi_trips_2months_sample.csv.gz', compression='gzip')